# Any and all pacakges used for cleaning and modeling

In [1]:
import pandas as pd
# !pip install wrds
import wrds
import os
import zipfile

c:\Users\ryant\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Part 1: Merging SPAC and IPO data
- Key Points: Self cleaned excel files with offering dates and firm names
- Both read in seperately and merged on the date variabe

In [98]:
spac_data = pd.read_csv("inputs/SPACs2016-2021.csv", header=0)
IPO_data = pd.read_csv("inputs/IPO_data.csv", header=0)


In [99]:
IPO_data


,Unnamed: 0,offer date,IPO name,CUSIP
0,0,01/30/1975,ROYSTER,78088610
1,1,06/09/1975,VARCO,92212610
2,2,06/10/1975,COORS ADOLPH,21701610
3,3,07/15/1975,KEYSTONE FOODS CORP,49348410
4,4,08/26/1975,C. F. BRAUN,10564710
...,...,...,...,...
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109
15443,15443,12/15/2023,Bayview Acquisition,NaN
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104
15445,15445,12/19/2023,Linkage Global Inc,G5500B102


In [97]:
# # Convert 'offer date' from 'YYYYMMDD' to 'MM/DD/YYYY'
# IPO_data['offer date'] = pd.to_datetime(IPO_data['offer date'],format='%Y%m%d')
# IPO_data['offer date'] = IPO_data['offer date'].dt.strftime('%m/%d/%Y')

# IPO_data.to_csv("inputs/IPO_data.csv")



In [100]:
IPO_data.rename(columns={'offer date' : 'Date of IPO'}, inplace=True)

In [108]:
merged_dates_df = pd.merge(spac_data ,IPO_data, on='Date of IPO', how='inner')

merged_dates_df.to_csv('inputs/merged_dates.csv')

In [103]:
#Reading in ccm data
ccm_youngfirms = "inputs/ccm_youngfirms_2000_2018.dta/ccm_youngfirms_2000_2018.dta"

if not os.path.exists(ccm_youngfirms):
    zip_path = "inputs/ccm_youngfirms_2000_2018.zip"

    with zipfile.ZipFile(zip_path,'r') as zip_ref:
        zip_ref.extractall("inputs/ccm_youngfirms_2000_2018.dta")
        
ccm_data = data = pd.read_stata(ccm_youngfirms)

In [109]:
ipo_data = pd.read_csv("inputs/IPO-age(9).csv")

merged_dates_df.rename(columns={'CUSIP' : 'cusip'}, inplace=True)
merge_df_ccm = pd.merge(merged_dates_df, ccm_data, on='cusip', how='inner')

merge_df_ccm

KeyError: 'cusip'